In [132]:
import os,math
import numpy as np
import pandas as pd
from tqdm.contrib import itertools
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D  # needed to plot 3-D surfaces

# tells matplotlib to embed plots within the notebook
%matplotlib inline
from sklearn.model_selection import train_test_split

In [6]:
order=pd.read_table('training_set_features.csv',sep=',')

In [9]:
order.shape

(26707, 36)

In [29]:
order.head()

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [11]:
order.isna().sum()

respondent_id                      0
xyz_concern                       92
xyz_knowledge                    116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_xyz                 2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_xyz_vacc_effective       391
opinion_xyz_risk                 388
opinion_xyz_sick_from_vacc       395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
m

In [12]:
parameters=pd.read_table('training_set_labels.csv',sep=',')

In [13]:
parameters.head(10)

,respondent_id,xyz_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0
5,5,0,0
6,6,0,0
7,7,1,1
8,8,0,0
9,9,0,0


In [46]:
df1=order.set_index('respondent_id')
df2=parameters.set_index('respondent_id')
result = df1.join(df2, how='left')
result.head(10)

,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_xyz,...,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,xyz_vaccine,seasonal_vaccine
respondent_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0,0
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,0,1
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,0,0
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,0,1
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,0,0
5,3.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,Own,Employed,atmpeygn,"MSA, Principle City",2.0,3.0,saaquncn,vlluhbov,0,0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Own,Employed,qufhixun,"MSA, Not Principle City",0.0,0.0,xicduogh,xtkaffoo,0,0
7,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,Own,Employed,bhuqouqj,Non-MSA,2.0,0.0,pxcmvdjn,xqwwgdyp,1,1
8,0.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,Own,Employed,bhuqouqj,"MSA, Not Principle City",1.0,0.0,xicduogh,ccgxvspp,0,0


In [47]:
result.drop(columns=['employment_industry', 'employment_occupation','health_insurance'], inplace=True)

In [72]:
result.hhs_geo_region.value_counts()

lzgpxyit    4297
fpwskwrf    3265
qufhixun    3102
oxchjgsf    2859
kbazzjca    2858
bhuqouqj    2846
mlyzmhmf    2243
lrircsnp    2078
atmpeygn    2033
dqpwygqj    1126
Name: hhs_geo_region, dtype: int64

In [76]:
result.replace({'age_group':{'18 - 34 Years':0,'35 - 44 Years':1,'45 - 54 Years':2,'55 - 64 Years':3,'65+ Years':4}},inplace=True)

In [66]:
result.replace({'education':{'< 12 Years':0,'12 Years':1,'Some College':2,'College Graduate':3}},inplace=True)

In [69]:
result.replace({'race':{'White':0,'Black':1,'Hispanic':2,'Other or Multiple':3}},inplace=True)

In [70]:
result.replace({'sex':{'Male':0,'Female':1}},inplace=True)

In [73]:
result.replace({'hhs_geo_region':{'lzgpxyit':0,'fpwskwrf':1,'qufhixun':2,'oxchjgsf':3,'kbazzjca':4,'bhuqouqj':5,'mlyzmhmf':6,'lrircsnp':7,'atmpeygn':8,'dqpwygqj':9}},inplace=True)

In [78]:
result.to_csv('results_putput.csv',index=False)

In [86]:
cleaned=result.dropna(axis=0)

In [105]:
cleaned.shape

(19642, 34)

In [95]:
def logistic(x):

    y = 1 / (1 + np.exp(-x))
    return y

In [96]:
def log_loss(y, y_dash):

    loss = - (y * np.log(y_dash)) - ((1 - y) * np.log(1 - y_dash))
    return loss

In [91]:
def cost_func_vec(y, y_dash):

    assert len(y) == len(y_dash), 
    m = len(y)
    loss_vec = np.array([log_loss(y[i], y_dash[i]) for i in range(m)])
    cost = np.dot(loss_vec, np.ones(m)) / m
    return cost

In [92]:
def cost_logreg_vec(X, y, w, b):

    m, n = X.shape
    assert len(y) == m, 
    assert len(w) == n, 
    z = np.matmul(X, w) + (b * np.ones(m))
    y_dash = logistic(z)
    cost = cost_func_vec(y, y_dash)
    return cost

In [93]:
def grad_logreg_vec(X, y, w, b): 

    m, n = X.shape
    assert len(y) == m, "Number of feature observations and number of target observations do not match"
    assert len(w) == n, "Number of features and number of weight parameters do not match"
    y_dash = logistic(np.matmul(X, w) + b * np.ones(m))
    grad_w = np.matmul(y_dash - y, X) / m
    grad_b = np.dot(y_dash - y, np.ones(m)) / m
    return grad_w, grad_b

In [99]:
def grad_desc(X, y, w, b, alpha, n_iter, show_cost = True, show_params = False): 
 
   
    m, n = X.shape
    assert len(y) == m, "Number of feature observations and number of target observations do not match"
    assert len(w) == n, "Number of features and number of weight parameters do not match"
    cost_history, params_history = [], []
    for i, j in itertools.product(range(n_iter), range(1)):
        grad_w, grad_b = grad_logreg_vec(X, y, w, b)   
        w += - alpha * grad_w
        b += - alpha * grad_b
        cost =  cost_logreg_vec(X, y, w, b)
        cost_history.append(cost)
        params_history.append([w, b])
        if show_cost == True and show_params == False and (i % math.ceil(n_iter / 10) == 0 or i == n_iter - 1):
            print(f"Iteration {i:6}:    Cost  {float(cost_history[i]):3.4f}")
        if show_cost == True and show_params == True and (i % math.ceil(n_iter / 10) == 0 or i == n_iter - 1):
            print(f"Iteration {i:6}:    Cost  {float(cost_history[i]):3.4f},    Params  {params_history[i]}")
    return w, b, cost_history, params_history


In [100]:
data_xyz=cleaned.drop(columns=['doctor_recc_seasonal','opinion_seas_vacc_effective','opinion_seas_risk','opinion_seas_sick_from_vacc','seasonal_vaccine'])

In [106]:
data_seas=cleaned.drop(columns=['doctor_recc_xyz','opinion_xyz_vacc_effective','opinion_xyz_risk','opinion_xyz_sick_from_vacc','xyz_vaccine','xyz_knowledge','xyz_concern'])

In [119]:
for col in data_xyz.columns:
    if (data_xyz[col].dtypes == 'int64' or data_xyz[col].dtypes == 'float64') and data_xyz[col].nunique() > 1:
        data_xyz[col] = (data_xyz[col] - data_xyz[col].min()) / (data_xyz[col].max() - data_xyz[col].min())
for col in data_seas.columns:
    if (data_seas[col].dtypes == 'int64' or data_seas[col].dtypes == 'float64') and data_seas[col].nunique() > 1:
        data_seas[col] = (data_seas[col] - data_seas[col].min()) / (data_seas[col].max() - data_seas[col].min())

In [121]:
X_xyz , y_xyz=data_xyz.drop(columns=['xyz_vaccine']),data_xyz['xyz_vaccine']

In [122]:
X_seas , y_seas=data_seas.drop(columns=['seasonal_vaccine']),data_seas['seasonal_vaccine']

In [125]:
X_xyz.shape

(19642, 28)

In [126]:
w_xyz=np.ones(28).astype(float)
b_xyz=1

In [129]:
X_seas.shape

(19642, 26)

In [130]:
w_seas=np.ones(26).astype(float)
b_seas=1

In [134]:
w_out_xyz, b_out_xyz, cost_history_xyz, params_history_xyz = grad_desc(X_xyz.to_numpy(),
                                                       y_xyz.to_numpy(),
                                                       w = w_xyz, # np.zeros(X_train.shape[1]),
                                                       b = b_xyz, # 0,
                                                       alpha = 0.1,
                                                       n_iter = 20000)

  0%|          | 0/20000 [00:00<?, ?it/s]

Iteration      0:    Cost  1.3229
Iteration   2000:    Cost  0.4132
Iteration   4000:    Cost  0.4038
Iteration   6000:    Cost  0.4001
Iteration   8000:    Cost  0.3985
Iteration  10000:    Cost  0.3977
Iteration  12000:    Cost  0.3974
Iteration  14000:    Cost  0.3972
Iteration  16000:    Cost  0.3972
Iteration  18000:    Cost  0.3971
Iteration  19999:    Cost  0.3971


In [135]:
w_out_seas, b_out_seas, cost_history_seas, params_history_seas = grad_desc(X_seas.to_numpy(),
                                                       y_seas.to_numpy(),
                                                       w = w_seas, # np.zeros(X_train.shape[1]),
                                                       b = b_seas, # 0,
                                                       alpha = 0.1,
                                                       n_iter = 20000)

  0%|          | 0/20000 [00:00<?, ?it/s]

Iteration      0:    Cost  5.5380
Iteration   2000:    Cost  0.4852
Iteration   4000:    Cost  0.4798
Iteration   6000:    Cost  0.4782
Iteration   8000:    Cost  0.4777
Iteration  10000:    Cost  0.4775
Iteration  12000:    Cost  0.4774
Iteration  14000:    Cost  0.4774
Iteration  16000:    Cost  0.4774
Iteration  18000:    Cost  0.4774
Iteration  19999:    Cost  0.4774


In [137]:
test_set=pd.read_table('test_set_features.csv',sep=',')

In [138]:
test_set.drop(columns=['employment_industry', 'employment_occupation','health_insurance'], inplace=True)

In [140]:
test_set.replace({'marital_status':{'Married':1,'Not Married':0}},inplace=True)

In [144]:
test_set..value_counts()

<= $75,000, Above Poverty    12775
> $75,000                     6863
Below Poverty                 2573
Name: income_poverty, dtype: int64

In [145]:
test_set.replace({'income_poverty':{'<= $75,000, Above Poverty':2,'> $75,000':1,'Below Poverty':0}},inplace=True)

In [146]:
test_set.replace({'rent_or_own':{'Rent':1,'Own':0}},inplace=True)

In [147]:
test_set.replace({'age_group':{'18 - 34 Years':0,'35 - 44 Years':1,'45 - 54 Years':2,'55 - 64 Years':3,'65+ Years':4}},inplace=True)

In [149]:
test_set.replace({'education':{'< 12 Years':0,'12 Years':1,'Some College':2,'College Graduate':3}},inplace=True)

In [150]:
test_set.replace({'race':{'White':0,'Black':1,'Hispanic':2,'Other or Multiple':3}},inplace=True)

In [151]:
test_set.replace({'sex':{'Male':0,'Female':1}},inplace=True)

In [152]:
test_set.replace({'hhs_geo_region':{'lzgpxyit':0,'fpwskwrf':1,'qufhixun':2,'oxchjgsf':3,'kbazzjca':4,'bhuqouqj':5,'mlyzmhmf':6,'lrircsnp':7,'atmpeygn':8,'dqpwygqj':9}},inplace=True)

In [156]:
test_set.census_msa.value_counts()

MSA, Not Principle  City    11926
MSA, Principle City          7805
Non-MSA                      6977
Name: census_msa, dtype: int64

In [155]:
test_set.replace({'employment_status':{'Unemployed':0,'Employed':1,'Not in Labor Force':2}},inplace=True)

In [157]:
test_set.replace({'census_msa':{'Non-MSA':0,'MSA, Not Principle  City':1,'MSA, Principle City':2}},inplace=True)

In [160]:
test_xyz=test_set.drop(columns=['doctor_recc_seasonal','opinion_seas_vacc_effective','opinion_seas_risk','opinion_seas_sick_from_vacc'])

In [161]:
test_seas=test_set.drop(columns=['doctor_recc_xyz','opinion_xyz_vacc_effective','opinion_xyz_risk','opinion_xyz_sick_from_vacc','xyz_knowledge','xyz_concern'])

In [166]:
test_xyz.set_index('respondent_id',inplace=True)

In [167]:
test_seas.set_index('respondent_id',inplace=True)

In [178]:
y_test_xyz=logistic(np.matmul(test_xyz.to_numpy(), w_out_xyz) + (b_out_xyz * np.ones(test_xyz.shape[0])))

In [179]:
y_test_seas=logistic(np.matmul(test_seas.to_numpy(), w_out_seas) + (b_out_seas * np.ones(test_seas.shape[0])))

In [177]:
for col in test_xyz.columns:
    if (test_xyz[col].dtypes == 'int64' or test_xyz[col].dtypes == 'float64') and test_xyz[col].nunique() > 1:
        test_xyz[col] = (test_xyz[col] - test_xyz[col].min()) / (test_xyz[col].max() - test_xyz[col].min())
for col in test_seas.columns:
    if (test_seas[col].dtypes == 'int64' or test_seas[col].dtypes == 'float64') and test_seas[col].nunique() > 1:
        test_seas[col] = (test_seas[col] - test_seas[col].min()) / (test_seas[col].max() - test_seas[col].min())

In [180]:
y_test_xyz

array([0.08301824, 0.04526438, 0.47005349, ..., 0.11786379, 0.05323553,
              nan])

In [190]:
df=pd.read_table('test_set_features.csv',sep=',')

In [193]:
df=df.iloc[0:,0:1]

In [195]:
df['xyz_vaccine']=y_test_xyz

In [196]:
df['seasonal_vaccine']=y_test_seas

In [197]:
df

,respondent_id,xyz_vaccine,seasonal_vaccine
0,26707,0.083018,0.182995
1,26708,0.045264,0.041221
2,26709,0.470053,0.651138
3,26710,0.463651,0.903381
4,26711,0.183584,0.595468
...,...,...,...
26703,53410,NaN,NaN
26704,53411,0.065663,0.257544
26705,53412,0.117864,0.184747
26706,53413,0.053236,0.492974


In [ ]:
df.to_csv('result_datahack.csv',)